In [2]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)

In [16]:
API_Token = "780ed9d1-727c-410f-b73e-2bed8b05c11a"
API_Secret = "9bdf99f4de7a672d0768761df407ab60"

In [17]:
BASE_URL = "https://carapi.app/api"

def get_jwt_token():
    login_url = f"{BASE_URL}/auth/login"
    credentials = {
        "api_token" : API_Token,
        "api_secret" : API_Secret
    }
    response = requests.post(login_url,json=credentials)
    response.raise_for_status()
    return response.text

jwt_token = get_jwt_token()

In [18]:
def get_trim_ids_for_make_model(make, model, year):
    url = f"{BASE_URL}/trims"
    headers = {'Authorization': f'Bearer {jwt_token}'}
    params = {'make': make, 'model': model, 'year': year}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()

    trim_ids = [trim['id'] for trim in data['data']]  # Adjusted to access the 'trims' key

    return trim_ids


def get_trim_details(trim_id):
    url = f"{BASE_URL}/trims/{trim_id}"
    headers = {'Authorization': f'Bearer {jwt_token}'}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    return response.json()


In [35]:
df = pd.read_csv("merged_dataset.csv")

api_data = {}

# Iterate over each row
for index, row in df.iterrows():
    car_name = row['model']
    year = pd.to_datetime(row['reg_date']).year if pd.notnull(row['reg_date']) else None
    
    words = car_name.split()
    if len(words) > 1:
        car_brand = words[0].lower()  # Convert to lowercase
        car_model = words[1].lower()  # Convert to lowercase
        car_details = None
        
        # Check if API data already exists
        if (car_brand, car_model, year) in api_data:
            car_details = api_data[(car_brand, car_model, year)]
        else:
            # Otherwise, make API call and store data in dictionary
            car_ids = get_trim_ids_for_make_model(car_brand , car_model, year)
            if car_ids:
                car_details = get_trim_details(car_ids[0])
                api_data[(car_brand, car_model, year)] = car_details
            else:
                api_data[(car_brand, car_model, year)] = None
        
        # Update dataframe with API data
        if car_details is not None:
            df.at[index, 'model_make_id'] = car_details['make_model_id']
            df.at[index, 'model_name'] = car_details['make_model']['name']
            df.at[index, 'model_trim'] = car_details['id']
            df.at[index, 'model_year'] = car_details['year']
            df.at[index, 'model_body'] = car_details['make_model_trim_body']['type']
            df.at[index, 'model_seats'] = car_details['make_model_trim_body']['seats']
            df.at[index, 'model_weight_kg'] = car_details['make_model_trim_body']['curb_weight']
            df.at[index, 'model_engine_fuel'] = car_details['make_model_trim_engine']['fuel_type']
            df.at[index, 'model_engine_cyl'] = car_details['make_model_trim_engine']['cylinders']
            df.at[index, 'model_drive'] = car_details['make_model_trim_engine']['drive_type']
            df.at[index, 'model_transmission_type'] = car_details['make_model_trim_engine']['transmission']
            df.at[index, 'model_fuel_cap_l'] = car_details['make_model_trim_mileage']['fuel_tank_capacity']

In [30]:
# df = pd.read_csv("merged_dataset.csv")
# df= df.iloc[:200]




# df["model_make_id"] = pd.NA
# df["model_name"] = pd.NA
# df["model_trim"] = pd.NA
# df["model_year"] = pd.NA
# df["model_body"] = pd.NA
# df["model_engine_cyl"] = pd.NA
# df["model_engine_fuel"] = pd.NA
# df["model_drive"] = pd.NA
# df["model_transmission_type"] = pd.NA
# df["model_seats"] = pd.NA
# df["model_weight_kg"] = pd.NA
# df["model_fuel_cap_l"] = pd.NA


# for index, row in df.iterrows():
#     car_name = row['model']
#     year = pd.to_datetime(row['reg_date']).year if pd.notnull(row['reg_date']) else None #transforming coe registered date to year
#     words = car_name.split()
#     if len(words) > 1:
#         car_brand = words[0]
#         car_model = words[1]
#         car_ids = get_trim_ids_for_make_model(car_brand , car_model, year)
#         if car_ids:
#             car_details = get_trim_details(car_ids[0])
#             if car_details:
#                 df.at[index, 'model_make_id'] = car_details['make_model_id']
#                 df.at[index, 'model_name'] = car_details['make_model']['name']
#                 df.at[index, 'model_trim'] = car_details['id']
#                 df.at[index, 'model_year'] = car_details['year']
#                 df.at[index, 'model_body'] = car_details['make_model_trim_body']['type']
#                 df.at[index, 'model_seats'] = car_details['make_model_trim_body']['seats']
#                 df.at[index, 'model_weight_kg'] = car_details['make_model_trim_body']['curb_weight']
#                 df.at[index, 'model_engine_fuel'] = car_details['make_model_trim_engine']['fuel_type']
#                 df.at[index, 'model_engine_cyl'] = car_details['make_model_trim_engine']['cylinders']
#                 df.at[index, 'model_drive'] = car_details['make_model_trim_engine']['drive_type']
#                 df.at[index, 'model_transmission_type'] = car_details['make_model_trim_engine']['transmission']
#                 df.at[index, 'model_fuel_cap_l'] = car_details['make_model_trim_mileage']['fuel_tank_capacity']

In [43]:
import json

#transforming pound to kg, and galloons to litres
df['model_weight_kg'] = df['model_weight_kg'] / 2.205
df['model_fuel_cap_l'] = df['model_fuel_cap_l'] * 3.785

df.to_csv('carAPI_combined.csv', index=False)
file_path = "carAPI_data.json"

data_str_keys = {str(key): value for key, value in api_data.items()}

# Specify the file path where you want to save the JSON
file_path = "carAPI.json"

# Write dictionary to JSON file
with open(file_path, 'w') as json_file:
    json.dump(data_str_keys, json_file)

In [44]:
api_data

{('honda', 'crossroad', 2008): None,
 ('subaru', 'forester', 2016): {'id': 20620,
  'make_model_id': 354,
  'year': 2016,
  'name': '2.0XT Premium',
  'description': '2.0XT Premium 4dr SUV AWD (2.0L 4cyl Turbo CVT)',
  'msrp': 29195,
  'invoice': 27332,
  'created': '2023-06-29T21:04:44-04:00',
  'modified': '2023-06-29T21:04:44-04:00',
  'make_model_trim_interior_colors': [{'id': 75710,
    'make_model_trim_id': 20620,
    'name': 'Black, cloth',
    'rgb': '49,49,51'}],
  'make_model_trim_exterior_colors': [{'id': 217042,
    'make_model_trim_id': 20620,
    'name': 'Crystal Black Silica',
    'rgb': '0,0,0'},
   {'id': 217044,
    'make_model_trim_id': 20620,
    'name': 'Crystal White Pearl',
    'rgb': '254,254,254'},
   {'id': 217046,
    'make_model_trim_id': 20620,
    'name': 'Dark Gray Metallic',
    'rgb': '130,132,129'},
   {'id': 217043,
    'make_model_trim_id': 20620,
    'name': 'Ice Silver Metallic',
    'rgb': '148,148,148'},
   {'id': 217041,
    'make_model_trim_id'